<a href="https://colab.research.google.com/github/johanhoffman/methods-in-computational-science/blob/main/MICS_Vector_Spaces_YinengWang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: Introduction**
**Yineng Wang**

# **Abstract**

This laboratory implements several fundamental operations in linear algebra: scalar product, matrix-vector product, matrix-matrix product, Euclidean norm and Euclidean distance, and carries out tests to check the correctness of the implementations.

#**About the code**

This report is written by Yineng Wang, based on Johan Hoffman's template.

In [1]:
# Copyright (C) 2020,2021 Johan Hoffman (jhoffman@kth.se)
# Copyright (C) 2021 Yineng Wang (yineng@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# **Set up environment**

To have access to the neccessary modules you have to run this cell.

In [2]:
import math
import random
from random import gauss
import unittest

import numpy as np
from scipy.spatial import distance

random.seed(0)

# **Introduction**

This laboratory implements the following five operations in linear algebra: scalar product, matrix-vector product, matrix-matrix product, Euclidean norm and Euclidean distance. The implementations are based on the lecture notes.

# **Method**

The implementations use Python's built-in list to represent objects in linear algebra (vectors and matrices).

## 1. Scalar product

The scalar product of two vectors $x, y \in R^n$ is defined as

$(x, y) = \sum_{i=1}^n x_i y_i$.

In [3]:
def scalar_product(x, y):
    if len(x) != len(y):
        raise ValueError("The shapes of x and y are not aligned.")
    return sum(xi * yi for xi, yi in zip(x, y))

## 2. Matrix-vector product

The product of a matrix $A = (a_{ij}) \in R^{m \times n}$ and a vector $x \in R^n$ is a vector $b \in R^m$.

Each component of $b$, $b_i$, is the scalar product of $A$'s row vector $a_{i:}$ and $x$.

In [4]:
def mat_vec_product(A, x):
    m, n = len(A), len(A[0])
    if len(x) != n:
        raise ValueError("The shapes of A and x are not aligned.")
    return [scalar_product(a, x) for a in A]

## 3. Matrix-matrix product

The product of a matrix $A = (a_{ij}) \in R^{m \times n}$ and $B = (b_{ij}) \in R^{n \times p}$ is a matrix $C = (c_{ij}) \in R^{m \times p}$, where $c_{ij} = \sum_{k=1}^n a_{ik} b_{kj}$.

In [5]:
def mat_mat_product(A, B):
    m, n, p = len(A), len(A[0]), len(B[0])
    if len(B) != n:
        raise ValueError("The shapes of A and B are not aligned.")
    C = [[0 for _ in range(p)] for __ in range(m)]
    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    return C

## 4. Euclidean norm

The Euclidean norm of a vector $x \in R^n$ is defined as

$\Vert x \Vert = (x, x)^{1/2} = \sqrt{\sum_{i=1}^n x_i^2}$.

In [6]:
def euclidean_norm(x):
    return math.sqrt(scalar_product(x, x))

## 5. Euclidean distance

The Euclidean distance between two vectors $x, y \in R^n$ is defined as

$d(x, y) = \Vert x - y \Vert = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}$.

In [7]:
def euclidean_distance(x, y):
    if len(x) != len(y):
        raise ValueError("The shape of x and y are not aligned.")
    return euclidean_norm([xi - yi for xi, yi in zip(x, y)])

# **Results**

The tests are carried out in Python's `unittest` framework. The NumPy's implmentations are used as a comparison to make sure the results are correct. Specially, SciPy's impmentation of Euclidean distance is used. The test for each function consists of a case for small objects plus a case for large objects (vectors and matrices with more than 10,000 components).

Since there might be round-off errors, NumPy's `assert_almost_equal` is used, which checks up to 7 decimals by default, instead of `assert_equal`.

## 1. Scalar product

In [8]:
class TestScalarProduct(unittest.TestCase):
    def test_unaligned_input(self):
        x, y = [1, 2], [1, 2, 3]
        np.testing.assert_raises(ValueError, scalar_product, x, y)

    def test_small(self):
        x, y = [1.3, -2.2, 0], [4.214, 5, -6.9983]
        p1 = scalar_product(x, y)
        p2 = np.dot(x, y)
        np.testing.assert_almost_equal(p1, p2)

    def test_large(self):
        x = [gauss(0, 1) for _ in range(10000)]
        y = [gauss(0, 1) for _ in range(10000)]
        p1 = scalar_product(x, y)
        p2 = np.dot(x, y)
        np.testing.assert_almost_equal(p1, p2)

## 2. Matrix-vector product

In [9]:
class TestMatVecProduct(unittest.TestCase):
    def test_unaligned_input(self):
        A, x = [[0, 1], [1, 0], [1, 0]], [1, 2, 3]
        np.testing.assert_raises(ValueError, mat_vec_product, A, x)

    def test_small(self):
        A, x = [[0.3, 5, -1.4], [6.2, 0, -3.9]], [-2.3, 6, -0.99]
        p1 = mat_vec_product(A, x)
        p2 = np.dot(A, x)
        np.testing.assert_almost_equal(p1, p2)

    def test_large(self):
        A = [[gauss(0, 1) for _ in range(10000)] for __ in range(100)]
        x = [gauss(0, 1) for _ in range(10000)]
        p1 = mat_vec_product(A, x)
        p2 = np.dot(A, x)
        np.testing.assert_almost_equal(p1, p2)

## 3. Matrix-matrix product

In [10]:
class TestMatMatProduct(unittest.TestCase):
    def test_unaligned_input(self):
        A = [[0, 1], [1, 2], [2, 3]]
        B = [[1], [2], [3]]
        np.testing.assert_raises(ValueError, mat_mat_product, A, B)

    def test_small(self):
        A = [[3, 4.5, 7.91], [-6.05, 0.1, 9.53]]
        B = [[3.17, 0], [4.3, -2.7], [-1.333, -9.82]]
        p1 = mat_mat_product(A, B)
        p2 = np.dot(A, B)
        np.testing.assert_almost_equal(p1, p2)

    def test_large(self):
        A = [[gauss(0, 1) for _ in range(100)] for __ in range(200)]
        B = [[gauss(0, 1) for _ in range(500)] for __ in range(100)]
        p1 = mat_mat_product(A, B)
        p2 = np.dot(A, B)
        np.testing.assert_almost_equal(p1, p2)

## 4. Euclidean norm

In [11]:
class TestEuclideanNorm(unittest.TestCase):
    def test_small(self):
        x = [1, 0, 0, 6, 0]
        n1 = euclidean_norm(x)
        n2 = np.linalg.norm(x)
        np.testing.assert_almost_equal(n1, n2)

    def test_large(self):
        x = [gauss(0, 1) for _ in range(10000)]
        n1 = euclidean_norm(x)
        n2 = np.linalg.norm(x)
        np.testing.assert_almost_equal(n1, n2)

## 5. Euclidean distance

In [12]:
class TestEuclideanDistance(unittest.TestCase):
    def test_unaligned_input(self):
        x, y = [1, 2], [1, 2, 3]
        np.testing.assert_raises(ValueError, euclidean_distance, x, y)

    def test_small(self):
        x, y = [1, 4, -2.3, 0], [0, 7.4, 1.94, 4.02]
        d1 = euclidean_distance(x, y)
        d2 = distance.euclidean(x, y)
        np.testing.assert_almost_equal(d1, d2)
    
    def test_large(self):
        x = [gauss(0, 1) for _ in range(10000)]
        y = [gauss(0, 1) for _ in range(10000)]
        d1 = euclidean_distance(x, y)
        d2 = distance.euclidean(x, y)
        np.testing.assert_almost_equal(d1, d2)

## Conduct the tests

In [13]:
if __name__ == '__main__':
    unittest.main(argv=['no_arg'], exit=False)

..............
----------------------------------------------------------------------
Ran 14 tests in 4.928s

OK


# **Discussion**

All testing cases are passed. The results are expected, under the assumption that round-off errors less than or equal to 8 decimals are tolerated.